## Week 2 - Full Prototype for Technical Questions Answerer

 This notebook will implement a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models.

In [33]:
# imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [34]:
# Initialization
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [38]:
system_message = """
You are an expert technical question answerer specializing in data science, programming, 
and software engineering. Your goal is to provide clear, accurate, and practical answers 
to technical questions.

When answering:
- Break down complex concepts into understandable explanations
- Provide code examples when relevant, with comments explaining key parts
- Mention common pitfalls or best practices
- If a question is ambiguous, state your assumptions or ask for clarification
- For debugging questions, explain both the fix and why the error occurred
- Cite specific documentation or resources when helpful

Always prioritize accuracy and clarity over speed. If you're unsure about something, 
acknowledge the uncertainty rather than guessing.
"""

In [39]:
# Streaming chat funcion
def chat(model, history):
    messages = [{"role": "system", "content": system_message}]
    for h in history:
        messages.append({"role": h["role"], "content": h["content"]})

    stream = openai.chat.completions.create(
        model=model, 
        messages=messages,
        stream=True
    )

    response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            response += chunk.choices[0].delta.content
            yield history + [{"role": "assistant", "content": response}]

In [40]:
#Gradio Interface
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
    with gr.Row():
        message = gr.Textbox(label="Chat with AI Assistant: ")
        model_dropdown = gr.Dropdown(
            choices=["gpt-4.1-mini","gpt-4o-mini", "gpt-4o", "gpt-4-turbo"], 
            value="gpt-4.1-mini", 
            label="Select Model"
        ) 

    def handle_submit(user_message, chat_history):
        # Add user message to history
        chat_history = chat_history + [{"role": "user", "content": user_message}]
        return "", chat_history

    message.submit(
        handle_submit, 
        inputs=[message, chatbot], 
        outputs=[message, chatbot]
    ).then(
        chat, 
        inputs=[model_dropdown, chatbot],
        outputs=[chatbot]
    )

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.


### Concluding Remarks
In this exercise, we successfully built a working AI chatbot with Gradio that includes streaming responses and the ability to switch between different models. The implementation demonstrates how to create an interactive interface for LLM applications.